In [47]:
import torch
from ultralytics import YOLO
import cv2
import onnxruntime as ort

In [48]:
pt_path = 'D:\\ProgPrj\\Undead\\HollywoodForever\\notebooks\\yolo\\runs\\detect\\train23\\weights\\best.pt'
onnx_path = '/HollywoodForever/application/model_repository/1/model.onnx'

In [49]:
model_pt = YOLO(pt_path)

In [50]:
img_path = 'D:\\ProgPrj\\Undead\\HollywoodForever\\data\\datasets\\test\\images\\50428374_JPG.rf.cd066443baf1c14e0e4275fa128089e6.jpg'
img = cv2.imread(img_path)

In [51]:
results_pt = model_pt(img_path)

In [52]:
# Отобразить предсказание на изображении
annotated_img = results_pt[0].plot()  # results[0] – это первое предсказание

In [78]:
# Показать изображение с bboxes
cv2.imshow("Detection", annotated_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [28]:
for result in results_pt[0].boxes:
    x1, y1, x2, y2 = map(int, result.xyxy[0])  # Получить координаты bbox (верхний левый и нижний правый угол)

    # Обрезать изображение по координатам bbox
    cropped_img = img[y1:y2, x1:x2]

    # Показать обрезанное изображение
    cv2.imshow("Cropped Detection", cropped_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
"""
CUDA:
image 1/1 D:\ProgPrj\Undead\HollywoodForever\data\datasets\test\images\50428374_JPG.rf.cd066443baf1c14e0e4275fa128089e6.jpg: 640x640 1 licence, 71.0ms
Speed: 1.0ms preprocess, 71.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
"""


"""
CPU:
image 1/1 D:\ProgPrj\Undead\HollywoodForever\data\datasets\test\images\50428374_JPG.rf.cd066443baf1c14e0e4275fa128089e6.jpg: 640x640 1 licence, 86.0ms
Speed: 3.0ms preprocess, 86.0ms inference, 3.5ms postprocess per image at shape (1, 3, 640, 640)
"""

### ONNX

In [88]:
import onnxruntime as ort
import cv2
import numpy as np

# Пути к модели и изображению
onnx_path = '/HollywoodForever/application/model_repository/1/model.onnx'
img_path = 'D:\\ProgPrj\\Undead\\HollywoodForever\\data\\datasets\\test\\images\\50428374_JPG.rf.cd066443baf1c14e0e4275fa128089e6.jpg'

In [95]:
# Задаем CUDAExecutionProvider
session = ort.InferenceSession(onnx_path, providers=['CPUExecutionProvider'])

In [96]:
# Загрузить изображение
img = cv2.imread(img_path)
original_img = img.copy()  # Сохраним копию оригинального изображения для вывода результатов

In [97]:
# Предобработка изображения
input_shape = (640, 640)  # Размер входного изображения, зависящий от модели
img_resized = cv2.resize(img, input_shape)
img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)
img_normalized = img_rgb / 255.0
img_transposed = np.transpose(img_normalized, (2, 0, 1))  # Порядок каналов (C, H, W)
img_input = img_transposed.astype(np.float32)
img_input = np.expand_dims(img_input, axis=0)  # Добавляем размерность для batch

In [98]:
# Выполнить инференс
input_name = session.get_inputs()[0].name
outputs = session.run(None, {input_name: img_input})

# Постобработка вывода для YOLO
# Обычно выход YOLO включает координаты bbox и уверенность
# Предположим, что в `outputs` первый элемент — это bbox с форматами [x_min, y_min, x_max, y_max, confidence, class]
detections = outputs[0][0]  # Извлекаем предсказания

# Коэффициенты масштабирования
height_ratio = original_img.shape[0] / input_shape[1]
width_ratio = original_img.shape[1] / input_shape[0]

In [99]:
for detection in detections:
    confidence = detection[4]  # Уверенность
    if confidence > 0.5:  # Фильтрация по уверенности
        class_id = int(detection[5])
        x_min = int(detection[0] * width_ratio)
        y_min = int(detection[1] * height_ratio)
        x_max = int(detection[2] * width_ratio)
        y_max = int(detection[3] * height_ratio)

        # Нарисовать bounding box на изображении
        cv2.rectangle(original_img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
        label = f'Class {class_id}: {confidence:.2f}'
        cv2.putText(original_img, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


In [100]:
# Показать изображение с bbox
cv2.imshow("Detection", original_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [105]:
from ultralytics import YOLO
import cv2

# Загрузите ONNX модель через Ultralytics
model = YOLO('/HollywoodForever/application/model_repository/1/model.onnx')

# Выполните предсказание
results = model.predict(source='D:\\ProgPrj\\Undead\\HollywoodForever\\data\\datasets\\test\\images\\50428374_JPG.rf.cd066443baf1c14e0e4275fa128089e6.jpg', imgsz=640)

# Отобразите изображение с предсказанными bounding boxes
for result in results:
    image = result.plot()  # Используем plot() для получения изображения с нарисованными bounding boxes
    cv2.imshow("Predicted Image", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
